In [1]:
import time
start1 = time.time()
from demo2_utils import *
from crowning2_utils import *
import time

def interchange_axes(array, axis1, axis2):
    new_array = array.copy()
    new_array[:, [axis1, axis2]] = new_array[:, [axis2, axis1]]
    return new_array

def get_vertical_position(bounds, point, toothlib):
    lower_bound, upper_bound = bounds
    normal = np.array([0, 1, 0])  # 'zx' normal
    projected_point = np.array([point[0], 0, point[2]])
    dot_product = np.dot(normal, projected_point)
    toothlib_copy = toothlib.copy()
    if dot_product >= np.dot(normal, lower_bound):
        print('UPPER')
        toothlib_copy.rotate_y(90, point=toothlib.center, inplace=True).rotate_x(15, point=toothlib.center, inplace=True)
    else:
        print('LOWER')
        toothlib_copy.rotate_y(-90, point=toothlib.center, inplace=True).rotate_x(-15, point=toothlib.center, inplace=True)
    return toothlib_copy

def gen_move_coords(dst_coord, src_coord):
    offset_val = 1
    offset = np.array([0, offset_val, 0])
    return (dst_coord + offset) - src_coord 


def get_horizontal_position(bounds, point, toothlib):
    left_bound, right_bound = bounds
    normal = np.array([0, 0, 1])  # 'zx' normal
    projected_point = np.array([point[0], point[1], 0])
    dot_product = np.dot(normal, projected_point)
    toothlib_copy = toothlib.copy()
    if dot_product >= np.dot(normal, left_bound):
        # pass
        print('LEFT')
        # toothlib_copy.rotate_y(-90, point=toothlib_copy.center, inplace=True)
        toothlib_copy.rotate_y(180, point=toothlib_copy.center, inplace=True)#.rotate_z(-20, point=toothlib.center, inplace=True)
    else:
        print('RIGHT')
        # toothlib_copy.rotate_z(20, point=toothlib.center, inplace=True)
        pass
        
        # toothlib_copy.rotate_y(180, point=toothlib_copy.center, inplace=True)
    return toothlib_copy

def callback2(point):
    mesh = pv.Sphere(center=point, radius=0.009)
    p1_.add_mesh(mesh, 
                 # style='wireframe', 
                 color='r',
                line_width=20)
    p1_.add_point_labels(point, [f"{point[0]:.2f}, {point[1]:.2f}, {point[2]:.2f}"])

print(int(time.time()-start1),'s')
udx_dirpath = '/Users/nimbyx/Projects/CROWNING/cases'

1 s


In [2]:
start = time.time()
target_path_list = []
ct = 0
for i,case in enumerate(os.listdir(udx_dirpath)):
    ct += 1
    case_path = os.path.join(udx_dirpath, case)
    my_case_details = get_toothnum(case_path)
    if not my_case_details:
        print(f'To remove: {case_path}')
        # os.remove(case_path)
        continue
    if my_case_details[1] in [30, 31,32]:
    # if str(my_case_details[1]) == '1':
        target_path_list.append(case_path)

print(f'Filtered cases: {len(target_path_list)}\nTotal cases: {len(os.listdir(udx_dirpath))}')
print(f'{int(time.time()-start)}')

get_toothnum Exception: File is not a zip file
To remove: /Users/nimbyx/Projects/CROWNING/cases/20230905113623089_2748284_W142B.zip
get_toothnum Exception: File is not a zip file
To remove: /Users/nimbyx/Projects/CROWNING/cases/.DS_Store
Filtered cases: 55
Total cases: 267
98


In [8]:
target_idx = 0

In [14]:
#PROCESS
target_idx -= 1

start1 = time.time()
file_path = target_path_list[target_idx]
mesh_prep_fname, mesh_prep_3m = load_prep_from_zip(file_path)
mesh_anta_fname, mesh_anta_3m = load_anta_from_zip(file_path)
mesh_big, mesh_small = get_two_largest_meshes_3m(mesh_prep_3m)
my_case_details = get_toothnum(file_path)
tooth_filepath = os.path.join(tooth_dirpath_ottawa, f"{my_case_details[1]}.stl")
tooth_mesh = pv.read(tooth_filepath)
tooth_desi = get_designer_tooth(file_path)

# case when z plane is larger than x plane
largest_plane = get_largest_axis_idx(mesh_prep_3m.bounds)
print(f'largest plane: {largest_plane}')
if largest_plane in [0,2]:
    if largest_plane == 2:
        tooth_mesh1 = get_horizontal_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
    elif largest_plane == 0:
        tooth_mesh1 = get_vertical_position(mesh_prep_3m.bounds, mesh_small.centroid, tooth_mesh)
    tooth_mesh2 = tooth_mesh1.scale(np.full(3, 1.20))
    tooth_mesh3 = tooth_mesh2.translate(gen_move_coords(mesh_small.centroid, tooth_mesh2.center))
    my_crown = tooth_mesh3.boolean_difference(pv.wrap(mesh_small))
    
    cam_pos = 'xy'
    my_shape = (1,4)
    p1_ = pv.Plotter(shape=my_shape)
        
    p1_.subplot(0,0) #1st plot
    p1_.add_mesh(mesh_small)
    p1_.add_mesh(mesh_big)
    p1_.add_mesh(tooth_mesh3, 
                 color='w')
    p1_.add_text(f'{my_case_details[0]}', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    p1_.show_grid()

    p1_.subplot(0,1) #2nd plot
    p1_.add_mesh(pv.wrap(mesh_anta_3m), opacity=.8)
    # p1_.add_mesh(tooth_mesh3.translate(gen_move_coords(tooth_mesh3.center-np.array([0,0,1]), tooth_mesh3.center)), color='w')
    p1_.add_mesh(tooth_mesh3, color='w')
    p1_.add_text(f'{my_case_details[1]}', position='upper_right')
    p1_.add_text('antagonist', position='lower_right', font_size=10)
    p1_.camera_position=cam_pos
    
    p1_.subplot(0,2) #3rd plot
    p1_.add_mesh(my_crown, color='gold', opacity=.9)
    # p1_.add_points(tooth_edges, color='red', line_width=10)
    p1_.add_mesh(pv.wrap(mesh_small))
    p1_.add_text('auto', position='lower_right', font_size=10)
    # p1_.enable_surface_point_picking(callback=callback2, show_point=True)
    p1_.show_grid()
    p1_.camera_position=cam_pos

    p1_.subplot(0,3) #4th plot
    p1_.add_mesh(pv.wrap(mesh_small))
    p1_.add_mesh(tooth_desi, color='gold', opacity=.9)
    p1_.add_text('designer', position='lower_right', font_size=10)
    p1_.show_grid()
    p1_.camera_position=cam_pos

    p1_.show()
else:
    print('y-plane largest')
print(int(time.time()-start1), 's')

largest plane: 0
LOWER


Widget(value="<iframe src='http://localhost:51193/index.html?ui=P_0x2911a8070_7&reconnect=auto' style='width: …

3 s


# WORKSPACE